<a href="https://colab.research.google.com/github/mks999/DML_Economics/blob/main/DML_PLR_and_Feature_Inference_2_9_25_Hi_Semi_Lo_EMP_EQI_IMF_PanelMultiDiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#1.Package Installation and Setup

# Install required packages
!pip install doubleml scikit-learn pandas numpy openpyxl matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR, DoubleMLIRM
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("=== COMPREHENSIVE DML ANALYSIS: EXPORT QUALITY & EMPLOYMENT ===")
print("Integrated PLR-IRM Framework for Multiple Specifications")
print("Comparable with GMM and Driscoll-Kraay Results")
print("=" * 70)



=== COMPREHENSIVE DML ANALYSIS: EXPORT QUALITY & EMPLOYMENT ===
Integrated PLR-IRM Framework for Multiple Specifications
Comparable with GMM and Driscoll-Kraay Results


In [18]:
#2: Data Loading and Preparation

# Load data
file_path = '/content/imputed Hi Lo Semi EQ_IMF 31.8.25.xlsx'
df = pd.read_excel(file_path)

print("Data Overview:")
print(f"Shape: {df.shape}")
print(f"Countries: {df['country'].nunique()}")
print(f"Time period: {df['time'].min()}-{df['time'].max()}")
print("\nAvailable columns:")
print(df.columns.tolist())

# Basic data cleaning
df = df.sort_values(['ctryid', 'time']).reset_index(drop=True)

print(f"\nData after sorting: {df.shape}")
print("\nSample data:")
print(df.head(35))

Data Overview:
Shape: (660, 25)
Countries: 33
Time period: 1995-2014

Available columns:
['country', 'time', 'mhtx', 'rnd', 'tariff', 'mobile', 'uv_index', 'gfcf', 'fdi', 'lnwages', 'reer', 'd1', 'aggemp', 'ctryid', 'dmy_uvindex', 'dmy_mhtx', 'msch', 'internet', 'covid_dummy', 'L_aggemp', 'High Skill', 'Low Skill', 'EQI_IMF', 'dmy_EQIimf', 'Semi Skill']

Data after sorting: (660, 25)

Sample data:
    country  time     mhtx      rnd  tariff     mobile  uv_index     gfcf  \
0   Austria  1995  54.3446  1.53577  10.810    4.82598     75.77  27.1040   
1   Austria  1996  55.8184  1.58077   6.941    7.52331     62.18  26.4922   
2   Austria  1997  55.8653  1.65491   5.372   14.55610     60.50  26.3834   
3   Austria  1998  55.4894  1.73155   3.199   28.74800     61.43  26.2500   
4   Austria  1999  56.2713  1.84537   2.697   53.18720     76.14  26.2733   
5   Austria  2000  58.8933  1.88602   2.410   76.36300     67.60  25.9401   
6   Austria  2001  58.7735  1.99210   3.370   81.35950     6

In [19]:
# 3: Model Specifications Setup

# Define comprehensive model specifications
MODEL_SPECS = {
    'spec_1_gmm_nodummy': {
        'name': 'GMM Spec 1 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 1 without dummies'
    },
    'spec_1_gmm_ctrydummy': {
        'name': 'GMM Spec 1 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 1 with country dummies'
    },
    'spec_2_gmm_nodummy': {
        'name': 'GMM Spec 2 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 2 without dummies'
    },
    'spec_2_gmm_ctrydummy': {
        'name': 'GMM Spec 2 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 2 with country dummies'
    },
     'spec_3_gmm_nodummy': {
        'name': 'GMM Spec 3 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'msch'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 3 without dummies'
    },
    'spec_3_gmm_ctrydummy': {
        'name': 'GMM Spec 3 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'msch'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 3 with country dummies'
    },
      'spec_4_gmm_nodummy': {
        'name': 'GMM Spec 4 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 4 without dummies'
    },
    'spec_4_gmm_ctrydummy': {
        'name': 'GMM Spec 4 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 4 with country dummies'
    },
      'spec_5_gmm_nodummy': {
        'name': 'GMM Spec 5 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 5 with dummies'
    },
    'spec_5_gmm_ctrydummy': {
        'name': 'GMM Spec 5 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 5 with country dummies'
    },
      'spec_6_gmm_nodummy': {
        'name': 'GMM Spec 6 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 6 without dummies'
    },
    'spec_6_gmm_ctrydummy': {
        'name': 'GMM Spec 6 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 6 with country dummies'
    },
     'spec_7_gmm_nodummy': {
        'name': 'GMM Spec 7 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 7 without dummies'
    },
    'spec_7_gmm_ctrydummy': {
        'name': 'GMM Spec 7 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 7 with country dummies'
    }
}


# Define analysis framework
DEPENDENT_VARS = {
    'High Skill': 'High Skill Employment',
    'Low Skill': 'Low Skill Employment',
    'Semi Skill': 'Semi Skill Employment'
}

TREATMENT_VARS = {
    'EQI_IMF': 'Export Quality Indicator (IMF)'
}

print("MODEL SPECIFICATIONS:")
print("=" * 50)
for spec_id, spec in MODEL_SPECS.items():
    print(f"{spec_id}: {spec['name']}")
    print(f"  Description: {spec['description']}")
    print(f"  Controls: {len(spec['controls'])} variables")
    print(f"  Dummies: {spec['dummies']}")
    print()

print("ANALYSIS FRAMEWORK:")
print(f"Dependent Variables: {list(DEPENDENT_VARS.keys())}")
print(f"Treatment Variables: {list(TREATMENT_VARS.keys())}")
print(f"Total Combinations: {len(DEPENDENT_VARS) * len(TREATMENT_VARS) * len(MODEL_SPECS)}")

MODEL SPECIFICATIONS:
spec_1_gmm_nodummy: GMM Spec 1 (No Dummies)
  Description: Comparable to GMM Spec 1 without dummies
  Controls: 7 variables
  Dummies: []

spec_1_gmm_ctrydummy: GMM Spec 1 (Country Dummies)
  Description: Comparable to GMM Spec 1 with country dummies
  Controls: 7 variables
  Dummies: ['dmy_treatment']

spec_2_gmm_nodummy: GMM Spec 2 (No Dummies)
  Description: Comparable to GMM Spec 2 without dummies
  Controls: 8 variables
  Dummies: []

spec_2_gmm_ctrydummy: GMM Spec 2 (Country Dummies)
  Description: Comparable to GMM Spec 2 with country dummies
  Controls: 8 variables
  Dummies: ['dmy_treatment']

spec_3_gmm_nodummy: GMM Spec 3 (No Dummies)
  Description: Comparable to GMM Spec 3 without dummies
  Controls: 5 variables
  Dummies: []

spec_3_gmm_ctrydummy: GMM Spec 3 (Country Dummies)
  Description: Comparable to GMM Spec 3 with country dummies
  Controls: 5 variables
  Dummies: ['dmy_treatment']

spec_4_gmm_nodummy: GMM Spec 4 (No Dummies)
  Description: Comp

In [20]:
# 4: Analysis Functions

def prepare_data_for_analysis(df, y_col, treatment_col, spec_name):
    """
    Prepare data for DML analysis based on specification
    """
    spec = MODEL_SPECS[spec_name]

    # Start with base controls
    x_cols = spec['controls'].copy()

    # Add dummies if specified
    if spec['dummies']:
        for dummy in spec['dummies']:
            if dummy == 'dmy_treatment':
                # Dynamically set treatment dummy - only include EQI_IMF
                if treatment_col == 'EQI_IMF':
                    x_cols.append('dmy_EQIimf')
                # Removed conditions for 'mhtx' and 'uv_index'
            else:
                x_cols.append(dummy)

    # Select required columns
    required_cols = [y_col, treatment_col] + x_cols

    # Check available columns
    available_cols = [col for col in required_cols if col in df.columns]
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        print(f"Warning: Missing columns {missing_cols}")
        print(f"Available columns: {available_cols}")

    # Use available columns
    df_clean = df[available_cols].copy() # Create a copy to avoid SettingWithCopyWarning

    # --- Data Cleaning: Convert relevant columns to numeric, handling errors ---
    cols_to_convert = [y_col, treatment_col] + [col for col in x_cols if col in df_clean.columns]
    for col in cols_to_convert:
        # Replace the problematic character '`' and convert to numeric
        if df_clean[col].dtype == 'object': # Only attempt replace on object type columns
            df_clean[col] = df_clean[col].astype(str).str.replace('`', '', regex=False)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    # --- End Data Cleaning ---


    df_clean.dropna(inplace=True) # Drop rows with NaN values after conversion

    actual_x_cols = [col for col in x_cols if col in df_clean.columns and col != treatment_col] # Ensure treatment is not in x_cols

    return df_clean, actual_x_cols, len(df) - len(df_clean)

def run_plr_analysis(df_clean, y_col, treatment_col, x_cols, spec_name):
    """
    Run PLR analysis
    """
    # Create DoubleML data object
    dml_data = DoubleMLData(df_clean, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

    # Define ML models
    ml_l = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
    ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)

    # Fit PLR model
    dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m, n_folds=5, n_rep=3)
    dml_plr.fit(store_models=True)

    return dml_plr

print("Analysis functions defined successfully!")
print("Functions available:")
print("- prepare_data_for_analysis(): Data preparation for each specification")
print("- run_plr_analysis(): PLR model estimation")


Analysis functions defined successfully!
Functions available:
- prepare_data_for_analysis(): Data preparation for each specification
- run_plr_analysis(): PLR model estimation


In [21]:
#5: Main Analysis Loop - PLR Results

# Initialize results storage
plr_results = {}
data_info = {}

print("=" * 70)
print("PLR ANALYSIS RESULTS")
print("=" * 70)

# Run analysis for each combination
for dep_var, dep_desc in DEPENDENT_VARS.items():
    if dep_var not in df.columns:
        print(f"Skipping {dep_var} - not available in dataset")
        continue

    for treat_var, treat_desc in TREATMENT_VARS.items():
        for spec_id, spec in MODEL_SPECS.items(): # Iterate through all new specifications

            print(f"\n{'='*50}")
            print(f"ANALYSIS: {dep_desc} ~ {treat_desc}")
            print(f"SPECIFICATION: {spec['name']}")
            print(f"{'='*50}")

            try:
                # Prepare data
                df_clean, x_cols, missing_obs = prepare_data_for_analysis(
                    df, dep_var, treat_var, spec_id
                )

                print(f"Data prepared: {len(df_clean)} obs, {missing_obs} dropped")
                print(f"Controls used: {x_cols}")

                # Run PLR
                dml_plr = run_plr_analysis(df_clean, dep_var, treat_var, x_cols, spec_id)

                # Store results
                key = f"{dep_var}_{treat_var}_{spec_id}"
                plr_results[key] = {
                    'model': dml_plr,
                    'coef': dml_plr.coef[0], # Access the scalar value
                    'se': dml_plr.se[0],     # Access the scalar value
                    'pval': dml_plr.pval[0],   # Access the scalar value
                    'ci_lower': dml_plr.coef[0] - 1.96 * dml_plr.se[0], # Access scalar values
                    'ci_upper': dml_plr.coef[0] + 1.96 * dml_plr.se[0], # Access scalar values
                    'n_obs': len(df_clean),
                    'dep_var': dep_var,
                    'treat_var': treat_var,
                    'spec_id': spec_id,
                    'controls': x_cols
                }

                data_info[key] = {
                    'df_clean': df_clean,
                    'x_cols': x_cols
                }

                # Display results - format the scalar values
                print(f"\nPLR RESULTS:")
                print(f"Coefficient: {plr_results[key]['coef']:.4f}")
                print(f"Std Error: {plr_results[key]['se']:.4f}")
                print(f"P-value: {plr_results[key]['pval']:.4f}")
                print(f"95% CI: [{plr_results[key]['ci_lower']:.4f}, {plr_results[key]['ci_upper']:.4f}]")

                significance = "***" if plr_results[key]['pval'] < 0.01 else "**" if plr_results[key]['pval'] < 0.05 else "*" if plr_results[key]['pval'] < 0.1 else ""
                print(f"Significance: {significance}")

            except Exception as e:
                print(f"Error in analysis: {str(e)}")
                continue

print(f"\n\nPLR ANALYSIS COMPLETED!")
print(f"Total successful analyses: {len(plr_results)}")

PLR ANALYSIS RESULTS

ANALYSIS: High Skill Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 1 (No Dummies)
Data prepared: 660 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer']

PLR RESULTS:
Coefficient: -2.3059
Std Error: 4.6761
P-value: 0.6219
95% CI: [-11.4711, 6.8593]
Significance: 

ANALYSIS: High Skill Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 1 (Country Dummies)
Data prepared: 660 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer', 'dmy_EQIimf']

PLR RESULTS:
Coefficient: 20.9602
Std Error: 6.6972
P-value: 0.0017
95% CI: [7.8338, 34.0867]
Significance: ***

ANALYSIS: High Skill Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 2 (No Dummies)
Data prepared: 660 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer']

PLR RESULTS:
Coefficient: 6.0817
Std Error: 4.2177
P-value: 0.1493
95% CI: [-2.1849, 

In [24]:
#9: Feature Importance Analysis

# Feature importance analysis for all models and skill types
print("=" * 80)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

def analyze_feature_importance(model, treatment_col, x_cols, model_type, dep_var_name):
    """Extract and display feature importance"""

    if model.models is not None:
        try:
            if model_type == 'PLR':
                fitted_ml_l = model.models['ml_l'][treatment_col][0][0]
                fitted_ml_m = model.models['ml_m'][treatment_col][0][0]

                print(f"\n{model_type} FEATURE IMPORTANCE:")

                # Outcome model (ml_l)
                if hasattr(fitted_ml_l, 'feature_importances_'):
                    # Ensure correct feature names are used based on x_cols and treatment_col
                    # Note: ml_l predicts the outcome (dependent variable), so treatment_col should NOT be included here
                    outcome_features = x_cols
                    if len(fitted_ml_l.feature_importances_) == len(outcome_features):
                        importance_l = pd.Series(
                            fitted_ml_l.feature_importances_,
                            index=outcome_features
                        ).sort_values(ascending=False)

                        print(f"\nOutcome Model ({dep_var_name} Prediction):") # Use dep_var_name
                        for var, imp in importance_l.head(5).items():
                            print(f"  {var}: {imp:.4f}")
                    else:
                         print(f"Warning: Feature importance length mismatch for Outcome Model (ml_l) in {dep_var_name} for {treatment_col}")


                # Treatment model (ml_m)
                if hasattr(fitted_ml_m, 'feature_importances_'):
                     if len(fitted_ml_m.feature_importances_) == len(x_cols):
                        importance_m = pd.Series(
                            fitted_ml_m.feature_importances_,
                            index=x_cols
                        ).sort_values(ascending=False)

                        print(f"\nTreatment Model ({treatment_col} Prediction):") # Dynamically get treatment variable name
                        for var, imp in importance_m.head(5).items():
                            print(f"  {var}: {imp:.4f}")
                     else:
                         print(f"Warning: Feature importance length mismatch for Treatment Model (ml_m) in {dep_var_name} for {treatment_col}")


        except Exception as e:
            print(f"Feature importance extraction failed: {e}")

# Analyze feature importance for all models and skill types
for key in plr_results.keys():
    plr = plr_results[key]

    print(f"\n{'='*60}")
    print(f"FEATURE IMPORTANCE: {plr['dep_var']} ~ {plr['treat_var']}")
    print(f"SPECIFICATION: {MODEL_SPECS[plr['spec_id']]['name']}")
    print(f"{'='*60}")

    # PLR feature importance
    analyze_feature_importance(plr['model'], plr['treat_var'], plr['controls'], 'PLR', plr['dep_var'])

FEATURE IMPORTANCE ANALYSIS

FEATURE IMPORTANCE: High Skill ~ EQI_IMF
SPECIFICATION: GMM Spec 1 (No Dummies)

PLR FEATURE IMPORTANCE:

Outcome Model (High Skill Prediction):
  gfcf: 0.2024
  tariff: 0.1859
  reer: 0.1679
  rnd: 0.1404
  msch: 0.1388

Treatment Model (EQI_IMF Prediction):
  rnd: 0.8089
  msch: 0.0707
  reer: 0.0310
  mobile: 0.0244
  gfcf: 0.0233

FEATURE IMPORTANCE: High Skill ~ EQI_IMF
SPECIFICATION: GMM Spec 1 (Country Dummies)

PLR FEATURE IMPORTANCE:

Outcome Model (High Skill Prediction):
  gfcf: 0.1774
  reer: 0.1748
  rnd: 0.1353
  dmy_EQIimf: 0.1292
  msch: 0.1272

Treatment Model (EQI_IMF Prediction):
  dmy_EQIimf: 0.6281
  msch: 0.1872
  rnd: 0.1162
  tariff: 0.0378
  gfcf: 0.0101

FEATURE IMPORTANCE: High Skill ~ EQI_IMF
SPECIFICATION: GMM Spec 2 (No Dummies)

PLR FEATURE IMPORTANCE:

Outcome Model (High Skill Prediction):
  lnwages: 0.1906
  reer: 0.1763
  gfcf: 0.1696
  msch: 0.1496
  rnd: 0.1439

Treatment Model (EQI_IMF Prediction):
  rnd: 0.7742
  lnwag

In [26]:
#10: Economic Interpretation and Policy Implications

# Economic interpretation and policy implications
print("=" * 80)
print("ECONOMIC INTERPRETATION AND POLICY IMPLICATIONS")
print("=" * 80)

def interpret_results(coef, treat_var, dep_var, method):
    """Provide economic interpretation of results"""

    if treat_var == 'EQI_IMF':
        treatment_name = "Export Quality Indicator (IMF)"
        unit = "unit increase"
    else:
        treatment_name = treat_var
        unit = "unit increase"


    effect_direction = "increases" if coef > 0 else "decreases"
    effect_magnitude = "large" if abs(coef) > 0.1 else "moderate" if abs(coef) > 0.05 else "small"

    # Get descriptive name for dependent variable
    dep_desc = DEPENDENT_VARS.get(dep_var, dep_var)

    interpretation = f"""
    A 1-{unit} in {treatment_name} {effect_direction} {dep_desc} by {abs(coef):.4f} percentage points.
    This represents a {effect_magnitude} {effect_direction[:-1]}ing effect estimated using {method}.
    """

    return interpretation.strip()

# Interpret key results
print("\nKEY FINDINGS:")
print("-" * 40)

# Interpret results for the first specification for each dependent variable
interpreted_keys = []
for dep_var in DEPENDENT_VARS.keys():
    for key, plr in plr_results.items():
        if plr['dep_var'] == dep_var and key.endswith('_spec_1_gmm_nodummy'):
            interpreted_keys.append(key)
            break # Only interpret the first specification for each dependent variable

for i, key in enumerate(interpreted_keys):
    plr = plr_results[key]

    dep_desc = DEPENDENT_VARS[plr['dep_var']]
    treat_desc = TREATMENT_VARS[plr['treat_var']]
    spec_name = MODEL_SPECS[plr['spec_id']]['name']

    print(f"\n{i+1}. {dep_desc} and {treat_desc} ({spec_name})")
    print("   " + "="*50)

    # PLR interpretation
    print(f"   PLR Result: {interpret_results(plr['coef'], plr['treat_var'], plr['dep_var'], 'PLR')}")


print(f"\n\nPOLICY IMPLICATIONS:")
print("-" * 30)
print("1. Export Quality Upgrading and Skilled Employment:")
print("   - Analyze coefficients for High Skill, Low Skill, and Semi Skill employment to understand differential impacts.")
print("   - Positive coefficients suggest export quality improvements boost specific skill levels of employment.")
print("   - Effect sizes indicate magnitude of policy impact for each skill group.")


print("\n2. Robustness:")
print("   - Compare results across different specifications to assess robustness.")
print("   - Multiple specifications ensure result stability.")

ECONOMIC INTERPRETATION AND POLICY IMPLICATIONS

KEY FINDINGS:
----------------------------------------

1. High Skill Employment and Export Quality Indicator (IMF) (GMM Spec 1 (No Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) decreases High Skill Employment by 2.3059 percentage points.
    This represents a large decreaseing effect estimated using PLR.

2. Low Skill Employment and Export Quality Indicator (IMF) (GMM Spec 1 (No Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) decreases Low Skill Employment by 4.8101 percentage points.
    This represents a large decreaseing effect estimated using PLR.

3. Semi Skill Employment and Export Quality Indicator (IMF) (GMM Spec 1 (No Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) decreases Semi Skill Employment by 14.4769 percentage points.
    This represents a large decreaseing effect estimated using PLR.


POLICY IMPLICATIONS:
-------------------------

### Average Feature Importance by Skill Type (Outcome Model)
This cell calculates and displays the average feature importance for the Outcome Model (ml_l) for each skill type across all specifications.

In [25]:
# Calculate Average Feature Importance

print("=" * 80)
print("AVERAGE FEATURE IMPORTANCE (OUTCOME MODEL)")
print("=" * 80)

# Dictionary to store feature importances for each skill type
skill_type_importances = {dep_var: [] for dep_var in DEPENDENT_VARS.keys()}

# Iterate through all PLR results
for key, plr in plr_results.items():
    dep_var = plr['dep_var']
    treatment_col = plr['treat_var']
    x_cols = plr['controls']
    model = plr['model']

    if model.models is not None:
        try:
            # Outcome model (ml_l)
            fitted_ml_l = model.models['ml_l'][treatment_col][0][0]

            if hasattr(fitted_ml_l, 'feature_importances_'):
                outcome_features = x_cols
                if len(fitted_ml_l.feature_importances_) == len(outcome_features):
                    importance_l = pd.Series(
                        fitted_ml_l.feature_importances_,
                        index=outcome_features
                    )
                    skill_type_importances[dep_var].append(importance_l)
                else:
                    print(f"Warning: Feature importance length mismatch for Outcome Model (ml_l) in {dep_var} for {treatment_col}. Skipping.")


        except Exception as e:
            print(f"Could not extract feature importance for {key}: {e}")
            continue

# Calculate average feature importance for each skill type
average_importances = {}
for dep_var, importances_list in skill_type_importances.items():
    if importances_list:
        # Concatenate all importance series for the skill type and calculate the mean
        all_importances = pd.concat(importances_list, axis=1, sort=False)
        average_importances[dep_var] = all_importances.mean(axis=1).sort_values(ascending=False)
    else:
        average_importances[dep_var] = pd.Series(dtype='float64') # Empty series if no importances

# Display average feature importance
for dep_var, avg_imp in average_importances.items():
    print(f"\nAverage Feature Importance for {DEPENDENT_VARS[dep_var]}:")
    if not avg_imp.empty:
        display(avg_imp)
    else:
        print("  No feature importance data available.")

AVERAGE FEATURE IMPORTANCE (OUTCOME MODEL)

Average Feature Importance for High Skill Employment:


,0
gfcf,0.185850
lnwages,0.177893
reer,0.176395
rnd,0.174423
dmy_EQIimf,0.160305
msch,0.149626
tariff,0.135628
internet,0.129003
fdi,0.077623
mobile,0.063513



Average Feature Importance for Low Skill Employment:


,0
msch,0.280633
dmy_EQIimf,0.257110
rnd,0.249997
gfcf,0.133186
reer,0.126843
lnwages,0.090099
mobile,0.078789
internet,0.073356
tariff,0.057449
fdi,0.040489



Average Feature Importance for Semi Skill Employment:


,0
rnd,0.302538
gfcf,0.225354
lnwages,0.173278
dmy_EQIimf,0.136998
msch,0.112294
tariff,0.094587
reer,0.088323
internet,0.078611
fdi,0.076053
mobile,0.058002
